In [ ]:
!pip install mlflow boto3 awscli optuna imbalanced-learn

In [ ]:
!aws configure

In [3]:
import mlflow
# Step 2: Set up the MLflow tracking server
mlflow.set_tracking_uri("http://ec2-100-26-36-125.compute-1.amazonaws.com:5000/")

In [4]:
# Set or create an experiment
mlflow.set_experiment("E ML Algos with HP Tuning")

<Experiment: artifact_location='s3://om-mlflow-bucket/622937322449081937', creation_time=1741368134336, experiment_id='622937322449081937', last_update_time=1741368134336, lifecycle_stage='active', name='E ML Algos with HP Tuning', tags={}>

In [5]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.svm import SVC
from imblearn.over_sampling import SMOTE
import mlflow
import mlflow.sklearn
import optuna

In [6]:
df=pd.read_csv('/content/reddit_processed.csv').dropna()

In [7]:
# Step 1: (Optional) Remapping - skipped since not strictly needed for SVM

# Step 2: Remove rows where the target labels (category) are NaN
df = df.dropna(subset=['category'])

X = df['clean_comment']
y = df['category']

# Step 5: Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Step 3: TF-IDF vectorizer setup
ngram_range = (1, 3)  # Trigram
max_features = 10000  # Set max_features to 1000
vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)

X_train=vectorizer.fit_transform(X_train)
X_test=vectorizer.transform(X_test)

# Step 4: Apply SMOTE to handle class imbalance
smote = SMOTE(random_state=42)
X_train, y_train = smote.fit_resample(X_train, y_train)

# Function to log results in MLflow
def log_mlflow(model_name, model, X_train, X_test, y_train, y_test):
    with mlflow.start_run():
        # Log model type
        mlflow.set_tag("mlflow.runName", f"{model_name}_SMOTE_TFIDF_Trigrams")
        mlflow.set_tag("experiment_type", "algorithm_comparison")

        # Log algorithm name as a parameter
        mlflow.log_param("algo_name", model_name)

        # Train model
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log the model
        mlflow.sklearn.log_model(model, f"{model_name}_model")


# Step 6: Optuna objective function for SVM
def objective_svm(trial):
    C = trial.suggest_float('C', 1e-4, 10.0, log=True)
    kernel = trial.suggest_categorical('kernel', ['linear', 'rbf', 'poly'])

    model = SVC(C=C, kernel=kernel, random_state=42)
    return accuracy_score(y_test, model.fit(X_train, y_train).predict(X_test))


# Step 7: Run Optuna for SVM, log the best model only
def run_optuna_experiment():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_svm, n_trials=30)

    # Get the best parameters and log only the best model
    best_params = study.best_params
    best_model = SVC(C=best_params['C'], kernel=best_params['kernel'], random_state=42)

    # Log the best model with MLflow, passing the algo_name as "SVM"
    log_mlflow("SVM", best_model, X_train, X_test, y_train, y_test)

# Run the experiment for SVM
run_optuna_experiment()


[I 2025-03-08 03:03:44,487] A new study created in memory with name: no-name-41d0fe72-8688-4dff-8a88-c5a88e76dfc1
[I 2025-03-08 03:07:52,223] Trial 0 finished with value: 0.5829810445929361 and parameters: {'C': 0.21343606340414642, 'kernel': 'poly'}. Best is trial 0 with value: 0.5829810445929361.
[I 2025-03-08 03:12:07,163] Trial 1 finished with value: 0.4346106641210964 and parameters: {'C': 0.0002342008363965265, 'kernel': 'poly'}. Best is trial 0 with value: 0.5829810445929361.
[I 2025-03-08 03:16:18,510] Trial 2 finished with value: 0.46215737078958136 and parameters: {'C': 0.03973679655252465, 'kernel': 'poly'}. Best is trial 0 with value: 0.5829810445929361.
[I 2025-03-08 03:20:27,422] Trial 3 finished with value: 0.4391108686758489 and parameters: {'C': 0.021734420981006763, 'kernel': 'poly'}. Best is trial 0 with value: 0.5829810445929361.
[I 2025-03-08 03:23:36,167] Trial 4 finished with value: 0.8126278467203054 and parameters: {'C': 0.28220776932505176, 'kernel': 'linear'}

🏃 View run SVM_SMOTE_TFIDF_Trigrams at: http://ec2-100-26-36-125.compute-1.amazonaws.com:5000/#/experiments/622937322449081937/runs/567d995512de478286f02dbd54c6a959
🧪 View experiment at: http://ec2-100-26-36-125.compute-1.amazonaws.com:5000/#/experiments/622937322449081937
